#### **Welcome to Assignment 5 on Deep Learning for Computer Vision.**
This notebook consists of two parts. In Part-1 you'll have to load pre-trained model and generate captions for a couple of Images, for Part-2 you'll have to implement an encoding layer from scratch. Detailed instructions are provided below

#### **Instructions**
1. Use Python 3.x to run this notebook
2. Write your code only in between the lines 'YOUR CODE STARTS HERE' and 'YOUR CODE ENDS HERE'.
you should not change anything else in the code cells, if you do, the answers you are supposed to get at the end of this assignment might be wrong.
3. Read documentation of each function carefully.
4. All the Best!


### Part-1

**Important:** Download `pretrained_model.zip` and `vocap.zip` files into your notebook root folder before you start the assignment. These are share in the following link: https://drive.google.com/drive/folders/12O5lTaofxKHcOlYeEEfzVwidUeEdVdAK?usp=sharing

The following cell automatically downloads them for you

In [1]:
## Leave this cell untouched!

!wget https://raw.githubusercontent.com/chentinghao/download_google_drive/master/download_gdrive.py
!python download_gdrive.py 18ZNReFUt7RtxixQjDknN-A4qMiQQP04i ./pretrained_model.zip
!python download_gdrive.py 1uIVoklkXeezsV35qXwZhA8eY0opSIgRA ./vocap.zip

--2021-10-02 12:10:18--  https://raw.githubusercontent.com/chentinghao/download_google_drive/master/download_gdrive.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1763 (1.7K) [text/plain]
Saving to: 'download_gdrive.py.1'

     0K .                                                     100%  224K=0.008s

2021-10-02 12:10:19 (224 KB/s) - 'download_gdrive.py.1' saved [1763/1763]


0.00B [00:00, ?B/s]
768kB [00:00, 7.72MB/s]
1.91MB [00:00, 8.61MB/s]
3.03MB [00:00, 6.57MB/s]
4.16MB [00:00, 7.57MB/s]
5.31MB [00:00, 8.48MB/s]
6.12MB [00:01, 2.94MB/s]
7.25MB [00:01, 3.79MB/s]
8.41MB [00:01, 4.76MB/s]
9.28MB [00:02, 2.59MB/s]
10.4MB [00:02, 3.38MB/s]
11.6MB [00:02, 4.31MB/s]
12.5MB [00:03, 2.06MB/s]
13.7MB [00:03, 2.73MB/s]
14.8MB [00:03, 3.56MB/s]
15.7MB [00:04, 

In [2]:
## Leave this cell untouched!
%%bash
mkdir models
unzip -q pretrained_model.zip -d models
unzip -q vocap.zip -d models
wget https://farm9.staticflickr.com/8526/8697076784_afb6c9ef22_z.jpg -O image3.jpg
wget https://farm5.staticflickr.com/4090/5200935390_bd4df7b05f_z.jpg -O image4.jpg


SyntaxError: invalid syntax (<ipython-input-2-8661cddd69a4>, line 3)

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np 
import argparse
import pickle 
import os
from PIL import Image
from torchvision import transforms 
import torch.nn as nn
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence

## Please DONOT remove these lines. 
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

In [ ]:
#### YOUR CODE STARTS HERE ####
# Check availability of GPU and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#### YOUR CODE ENDS HERE ####

In [ ]:
# this is a simple wrapper for dealing with the vocabulary! 
# this'll be automatically used when you load the provided vocabulary file
class Vocabulary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        #### YOUR CODE STARTS HERE ####
        # load the resnet152 network provided by torchvision
        resnet = 
        # remove the last FC layer in the network
        modules =
        #define a sequential model with the modules obtained above
        self.resnet =
        # define a linear layer with out_features as embed_size
        self.linear =
        # define a 1d batch norm layer with embed_size as the number of features and a momentum of 0.01 
        self.bn = 
        #### YOUR CODE ENDS HERE ####
    def forward(self, images):
        #### YOUR CODE STARTS HERE ####
        # calculate the forward pass in the order of base_model -> linear -> bn
        # reshape features, if necessary
        features = 
        #### YOUR CODE ENDS HERE ####
        return features


class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, max_seq_length=20):
        super(DecoderRNN, self).__init__()
        self.max_seg_length = max_seq_length
        #### YOUR CODE STARTS HERE ####
        # define an embedding layer with vocab_size as the size of the dictionary
        # and embed_size as the size of each embedding
        self.embed =
        # define a LSTM layer with embed_size, hidden_size, num_layers, set batch_first to True
        self.lstm =
        # define a linear layer with out features as the vocab size
        self.linear =
        #### YOUR CODE ENDS HERE ####

    def forward(self, features, captions, lengths):
        #### YOUR CODE STARTS HERE ####
        # get embeddings for the captions
        embeddings = 
        # concatenate the features with embeddings
        embeddings = 
        #### YOUR CODE ENDS HERE ####
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True) 
        hiddens, _ = self.lstm(packed)
        outputs = self.linear(hiddens[0])
        return outputs
    
    def sample(self, features, states=None):
        sampled_ids = []
        inputs = features.unsqueeze(1)
        for i in range(self.max_seg_length):
            hiddens, states = self.lstm(inputs, states)          # hiddens: (batch_size, 1, hidden_size)
            outputs = self.linear(hiddens.squeeze(1))            # outputs:  (batch_size, vocab_size)
            _, predicted = outputs.max(1)                        # predicted: (batch_size)
            sampled_ids.append(predicted)
            inputs = self.embed(predicted)                       # inputs: (batch_size, embed_size)
            inputs = inputs.unsqueeze(1)                         # inputs: (batch_size, 1, embed_size)
        sampled_ids = torch.stack(sampled_ids, 1)                # sampled_ids: (batch_size, max_seq_length)
        return sampled_ids

In [ ]:

def main(img_path):
    #### YOUR CODE STARTS HERE ####
    # Image preprocessing
    # define the transforms with normalization values: [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
    transform = transforms.Compose([
        transforms.ToTensor(), # convert to a tensor
        transforms.Normalize((0.485, 0.456, 0.406), # normalise the image
                             (0.229, 0.224, 0.225))])
    
    with open('./models/vocab.pkl', 'rb') as f: # load the vocaulary wrapper file
        vocab = pickle.load(f)

    # build encoder with a embed size of 256
    encoder =
    # build decoder with embed_size of 256, hidden_size of 512, vocab_size, 1 layer
    decoder = 

    # move the encoder and decoder to device
    encoder =
    decoder =
    
    # load the pre-trained weights for encoder and decoder
    encoder.
    decoder.
    #### YOUR CODE ENDS HERE ####

    # Prepare an image
    image = Image.open(img_path).convert('RGB')
    image = image.resize([224, 224], Image.LANCZOS)
    image = transform(image).unsqueeze(0)
    image_tensor = image.to(device)
    
    # Generate an caption from the image
    feature = encoder(image_tensor)
    sampled_ids = decoder.sample(feature)
    sampled_ids = sampled_ids[0].cpu().numpy()
    
    # Convert word_ids to words
    sampled_caption = []
    for word_id in sampled_ids:
        word = vocab.idx2word[word_id]
        sampled_caption.append(word)
        if word == '<end>':
            break
    sentence = ' '.join(sampled_caption)
    
    # Print out the image and the generated caption
    print (sentence)
    image = Image.open(img_path)
    plt.imshow(np.asarray(image))

In [ ]:
main('image3.jpg')

#### Question-1

Pick the caption displayed for image3.jpg
1. a man looking at his phone
2. a woman looking at his phone
3. many people in front of a building 
4. a man and woman are standing in front of a building

In [ ]:
main('image4.jpg')

#### Question-2

Pick the caption displayed for image4.jpg
1. a bunch of items that are on the floor
2. an umbrella with a basket that are on the sidewalk 
3. a bunch of vegetables that are inside a shop
4. a bunch of umbrellas that are sitting on a sidewalk 

### Part-2

This part is based on official PyTorch tutorial [Sequence-to-sequence modeling with nn.Transformer and TorchText](https://pytorch.org/tutorials/beginner/transformer_tutorial.html). Unlike the tutorial you'll have to code the encoding layer from scratch.

**Note:**  We use a different set of hyper-parameters from the ones used in tutorial, ensure you only use the code in this notebook while reporting the answers

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn as nn
import numpy as np

In [ ]:
# You might need to restart your Jupyter runtime after running this cell

!pip install torchtext==0.8.1 

In [ ]:
import random
import os
import torch
import numpy as np

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)  

set_seed(0)

In [ ]:
class CustomEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=256, dropout=0.3, activation="relu"):
        super(CustomEncoderLayer, self).__init__()
        #### YOUR CODE STARTS HERE ####
        # define a multi head attention layer d_model, nhead and dropout
        self.self_attn = 
        # define a linear layer with 512 as the number of output features
        self.linear1 =
        # define a dropout layer using the passed value
        self.dropout =
        # define a linear layer with dim_feedforward as the number of output features
        self.linear2 = 
        # define a linear layer with d_model as the number of output features
        self.linear3 =
        # define a normalization layer (not batch and instance norm) with d_model as the normalized shape
        self.norm1 =
        # define a normalization layer (not batch and instance norm) with d_model as the normalized shape
        self.norm2 =
        # define a dropout layer using the passed value
        self.dropout1 =
        # define a dropout layer using the passed value
        self.dropout2 =
        #### YOUR CODE ENDS HERE ####
        self.activation = nn.ReLU()
        
    def forward(self, src, src_mask, src_key_padding_mask):

        src2 = self.self_attn(src, src, src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        #### YOUR CODE STARTS HERE ####
        # use dropout1 of src2 and add to src
        src = 
        # use the norm1 layer on src
        src = 
        # use linear1 on src along with activation
        src2 =
        # use dropout on src2
        src2 =
        # use linear 2 on src2 along with activation
        src2 =
        # use linear3 on src2
        src2 = 
        # use dropout2 on src2 and add it to src
        src =
        # use norm2 on src
        src =
        #### YOUR CODE ENDS HERE ####
        return src

In [ ]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.1):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = CustomEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.3, max_len=3000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("spacy"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 64
eval_batch_size = 32
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

In [ ]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [ ]:
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 124 # embedding dimension
nhid = 64 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 1 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 4 # the number of heads in the multiheadattention models
dropout = 0.1 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
lr = 1.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        if data.size(0) != bptt:
            src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [ ]:
best_val_loss = float("inf")
epochs = 5 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()


#### Question-3

Report the final perplexity value (`ppl`) displayed (If you are not getting the exact number shown in options, please report the closest number).
1. 151.45
2. 174.21
3. 148.91
4. 123.99

#Part-3
# VAE
Our VAE implementation will consist solely of fully connected layers. We'll take the 1 x 28 x 28 shape of our input and flatten the features to create an input dimension size of 784.

In this section you'll define the Encoder and Decoder models, implement the reparametrization trick, forward pass, and loss function to train your first VAE.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import numpy as np 
import random
import argparse
import pickle 
import os
import math
from PIL import Image

import torch
from torch.nn import init
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data
from torch.utils.data import DataLoader
from torch.utils.data import sampler

from torchvision import transforms
from torchvision.utils import save_image
import torchvision.models as models
import torchvision.transforms as T
import torchvision.datasets as dset

%matplotlib inline

# for plotting
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['font.size'] = 16
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)  

set_seed(0)

In [ ]:
############################################################################################
#                                  BEGINNING OF YOUR CODE                                  #
############################################################################################
# TODO: Check availability of GPU and set the device accordingly                                 #
device = None
############################################################################################
#                                  END OF YOUR CODE                                  #
############################################################################################

In [ ]:
# Downloading the dataset

batch_size = 128

mnist_train = dset.MNIST('./MNIST_data', train=True, download=True,
                           transform=T.ToTensor())
loader_train = DataLoader(mnist_train, batch_size=batch_size,
                          shuffle=True, drop_last=True, num_workers=2)

In [ ]:
# Custom Function to show images

def show_images(images):
    images = torch.reshape(images, [images.shape[0], -1])  # images reshape to (batch_size, D)
    sqrtn = int(math.ceil(math.sqrt(images.shape[0])))
    sqrtimg = int(math.ceil(math.sqrt(images.shape[1])))

    fig = plt.figure(figsize=(sqrtn, sqrtn))
    gs = gridspec.GridSpec(sqrtn, sqrtn)
    gs.update(wspace=0.05, hspace=0.05)

    for i, img in enumerate(images):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(img.reshape([sqrtimg,sqrtimg]))
    return 

## Define the model parameters

1. Define the `encoder`, `decoder`, `mu_layer`, and `logvar_layer` in the initialization (`__init__` function) of the below class. Use nn.Sequential to define the encoder, and separate Linear layers for the mu and logvar layers. In all of these layers, H will be a hidden dimension you set and will be the same across all encoder and decoder layers. 

**Architecture for the encoder is described below:**


 * `Flatten` (Hint: nn.Flatten)
 * Fully connected layer with input size 784 (`input_size`) and output size H
 * `ReLU`
 * Fully connected layer with input_size H and output size H
 * `ReLU`
 * Fully connected layer with input_size H and output size H
 * `ReLU`


We'll now define the decoder, which will take the latent space representation and generate a reconstructed image. The architecture is as follows: 


 **Architecture for the decoder is described below:**


 * Fully connected layer with input size as the latent size (Z) and output size H
 * `ReLU`
 * Fully connected layer with input_size H and output size H
 * `ReLU`
 * Fully connected layer with input_size H and output size H
 * `ReLU`
 * Fully connected layer with input_size H and output size 784 (`input_size`)
 * `Sigmoid`
 * `Unflatten` (nn.Unflatten)


Please do not touch the `forward` function for now. We will come back to it later.

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_size, latent_size=15):
        super(VAE, self).__init__()
        self.input_size = input_size # H*W
        self.latent_size = latent_size # Z
        self.hidden_dim = 200

        self.encoder = None
        self.mu_layer = None
        self.logvar_layer = None
        self.decoder = None

        ############################################################################################
        #                                  BEGINNING OF YOUR CODE                                  #
        ############################################################################################
        ############################################################################################
        # TODO: Implement the fully-connected encoder architecture described in the notebook.      #
        # Specifically, self.encoder should be a network that inputs a batch of input images of    #
        # shape (N, 1, H, W) into a batch of hidden features of shape (N, H_d). Set up             #
        # self.mu_layer and self.logvar_layer to be a pair of linear layers that map the hidden    #
        # features into estimates of the mean and log-variance of the posterior over the latent    #
        # vectors; the mean and log-variance estimates will both be tensors of shape (N, Z).       #
        ############################################################################################
        # Replace "pass" statement with your code
        pass
        ############################################################################################
        # TODO: Implement the fully-connected decoder architecture described in the notebook.      #
        # Specifically, self.decoder should be a network that inputs a batch of latent vectors of  #
        # shape (N, Z) and outputs a tensor of estimated images of shape (N, 1, H, W).             #
        ############################################################################################
        # Replace "pass" statement with your code
        pass
        ############################################################################################
        #                                      END OF YOUR CODE                                    #
        ############################################################################################


    def forward(self, x):
        """
        Performs forward pass through FC-VAE model by passing image through 
        encoder, reparametrize trick, and decoder models
    
        Inputs:
        - x: Batch of input images of shape (N, 1, H, W)
        
        Returns:
        - x_hat: Reconstruced input data of shape (N,1,H,W)
        - mu: Matrix representing estimated posterior mu (N, Z), with Z latent space dimension
        - logvar: Matrix representing estimataed variance in log-space (N, Z), with Z latent space dimension
        """
        x_hat = None
        mu = None
        logvar = None

        ############################################################################################
        #                                  BEGINNING OF YOUR CODE                                  #
        ############################################################################################
        ############################################################################################
        # TODO: Implement the forward pass by following these steps                                #
        # (1) Pass the input batch through the encoder model, pass the output to mu_layer and      # 
        # logvar_layer to get the posterior mu and logvariance                                     #
        # (2) Reparametrize to compute  the latent vector z                                        #
        # (3) Pass z through the decoder to resconstruct x                                         #
        ############################################################################################
        # Replace "pass" statement with your code
        pass
        ############################################################################################
        #                                      END OF YOUR CODE                                    #
        ############################################################################################
        return x_hat, mu, logvar

## Reparameterization Trick

Now we'll apply a reparametrization trick in order to estimate the posterior $z$ during our forward pass, given the $\mu$ and $\sigma^2$ estimated by the encoder. A simple way to do this could be to simply generate a normal distribution centered at our  $\mu$ and having a std corresponding to our $\sigma^2$. However, we would have to backpropogate through this random sampling that is not differentiable. Instead, we sample initial random data $\epsilon$ from a fixed distrubtion, and compute $z$ as a function of ($\epsilon$, $\sigma^2$, $\mu$). Specifically:

$z = \mu + \sigma\epsilon$

We can easily find the partial derivatives w.r.t $\mu$ and $\sigma^2$ and backpropagate through $z$. If $\epsilon = \mathcal{N} (0,1)$, then its easy to verify that the result of our forward pass calculation will be a distribution centered at $\mu$ with variance $\sigma^2$.

Implement the `reparametrization` function.

In [ ]:
def reparametrize(mu, logvar):
    """
    Differentiably sample random Gaussian data with specified mean and variance using the
    reparameterization trick.
    Inputs:
    - mu: Tensor of shape (N, Z) giving means
    - logvar: Tensor of shape (N, Z) giving log-variances
    Returns: 
    - z: Estimated latent vector of shape (N, Z), where z[i, j] is a random value sampled from a Gaussian with
         mean mu[i, j] and log-variance logvar[i, j].
    """
    z = None

    ################################################################################################
    #                                  BEGINNING OF YOUR CODE                                      #
    ################################################################################################
    ################################################################################################
    # TODO: Reparametrize by initializing epsilon as a normal distribution and scaling by          #
    # posterior mu and sigma to estimate z                                                         #
    ################################################################################################
    # Replace "pass" statement with your code
    pass
    ################################################################################################
    #                              END OF YOUR CODE                                                #
    ################################################################################################
    return z

## Define the forward pass in the `VAE` class

Go back to the cell above the Reparameterization trick header, which contains the definition of the `VAE` class.

Implement the forward pass in the `forward` function, and run the cell.

## Loss function

The loss function for VAEs contains two terms: A reconstruction loss term (left) and KL divergence term (right). 

$-E_{Z~q_{\phi}(z|x)}[log p_{\theta}(x|z)] + D_{KL}(q_{\phi}(z|x), p(z)))$

Note that this is the negative of the variational lowerbound -this ensures that when we are minimizing this loss term, we're maximizing the variational lowerbound. The reconstruction loss term can be computed by simply using the binary cross entropy loss between the original input pixels and the output pixels of our decoder (Hint: `nn.functional.binary_cross_entropy`). The KL divergence term works to force the latent space distribution to be close to a prior distribution (we're using a standard normal gaussian as our prior).

To help you out, we've derived an unvectorized form of the KL divergence term for you.
Suppose that $q_\phi(z|x)$ is a $Z$-dimensional diagonal Gaussian with mean $\mu_{z|x}$ of shape $(Z,)$ and standard deviation $\sigma_{z|x}$ of shape $(Z,)$, and that $p(z)$ is a $Z$-dimensional Gaussian with zero mean and unit variance. Then we can write the KL divergence term as:

$D_{KL}(q_{\phi}(z|x), p(z))) = -\frac{1}{2} \sum_{j=1}^{J} (1 + log(\sigma_{z|x}^2)_{j} - (\mu_{z|x})^2_{j} - (\sigma_{z|x})^2_{j}$)

Implement a vectorized version of this loss that operates on minibatches.
You should average the loss across samples in the minibatch.

In [ ]:
def loss_function(x_hat, x, mu, logvar):
    """
    Computes the negative variational lower bound loss term of the VAE (refer to formulation in notebook).
    Inputs:
    - x_hat: Reconstruced input data of shape (N, 1, H, W)
    - x: Input data for this timestep of shape (N, 1, H, W)
    - mu: Matrix representing estimated posterior mu (N, Z), with Z latent space dimension
    - logvar: Matrix representing estimated variance in log-space (N, Z), with Z latent space dimension
    
    Returns:
    - loss: Tensor containing the scalar loss for the negative variational lowerbound
    """
    loss = None
    
    ################################################################################################
    #                                  BEGINNING OF YOUR CODE                                      #
    ################################################################################################
    ################################################################################################
    # TODO: Compute negative variational lowerbound loss as described in the notebook.             #
    # Note that the log variance is provided as the input to your function. Interpret the hints    #
    # above accordingly.
    ################################################################################################
    # Replace "pass" statement with your code
    pass    
    ################################################################################################
    #                            END OF YOUR CODE                                                  #
    ################################################################################################
    return loss

## Train the VAE

In [ ]:
# Leave this cell untouched
def train_vae(epoch, model, train_loader):
    """
    Train the VAE!

    Inputs:
    - epoch: Current epoch number 
    - model: VAE model object
    - train_loader: PyTorch Dataloader object that contains our training data
    """
    model.train()
    train_loss = 0
    num_classes = 10
    loss = None
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    for batch_idx, (data, labels) in enumerate(train_loader):
        data = data.to(device)
        recon_batch, mu, logvar = model(data)
        optimizer.zero_grad()
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.data
        optimizer.step()
    print('Train Epoch: {} \tLoss: {:.6f}'.format(
        epoch, loss.data))

In [ ]:
# Leave this cell untouched

set_seed(0)

num_epochs = 10
latent_size = 15
input_size = 28*28


vae_model = VAE(input_size, latent_size=latent_size)
vae_model.to(device)
for epoch in range(0, num_epochs):
  train_vae(epoch, vae_model, loader_train)

## Question 4

Report the final validation loss displayed after training the VAE, as displayed in the cell.

1. 106.054
2. 108.744
3. 115.499
4. 110.035

# GANs

We can think of GANs as a back and forth process of the generator ($G$) trying to fool the discriminator ($D$), and the discriminator trying to correctly classify real vs. fake as a minimax game:
$$\underset{G}{\text{minimize}}\; \underset{D}{\text{maximize}}\; \mathbb{E}_{x \sim p_\text{data}}\left[\log D(x)\right] + \mathbb{E}_{z \sim p(z)}\left[\log \left(1-D(G(z))\right)\right]$$
where $z \sim p(z)$ are the random noise samples, $G(z)$ are the generated images using the neural network generator $G$, and $D$ is the output of the discriminator, specifying the probability of an input being real. 
<br><br>
In [Goodfellow et al.](https://arxiv.org/abs/1406.2661), they analyze this minimax game and show how it relates to minimizing the Jensen-Shannon divergence between the training data distribution and the generated samples from $G$.

In [ ]:
import math
import torch
import torch.nn as nn
from torch.nn import init
import torchvision
import random
import os
import numpy as np
import torchvision.transforms as T
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)  

set_seed(0)

In [ ]:
def show_images(images):
    images = torch.reshape(images, [images.shape[0], -1])  # images reshape to (batch_size, D)
    sqrtn = int(math.ceil(math.sqrt(images.shape[0])))
    sqrtimg = int(math.ceil(math.sqrt(images.shape[1])))

    fig = plt.figure(figsize=(sqrtn, sqrtn))
    gs = gridspec.GridSpec(sqrtn, sqrtn)
    gs.update(wspace=0.05, hspace=0.05)

    for i, img in enumerate(images):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(img.reshape([sqrtimg,sqrtimg]))
    return 

In [ ]:
dtype = torch.float

############################################################################################
#                                  BEGINNING OF YOUR CODE                                  #
############################################################################################
# TODO: Check availability of GPU and set the device accordingly                                 #
device = None
############################################################################################
#                                  END OF YOUR CODE                                  #
############################################################################################

In [ ]:
# let us load the input images from the dataset and visualize some images!

batch_size = 128
NOISE_DIM = 96

print('download MNIST if not exist')

mnist_train = dset.MNIST('./MNIST_data', train=True, download=True,
                           transform=T.ToTensor())
loader_train = DataLoader(mnist_train, batch_size=batch_size,
                          shuffle=True, drop_last=True, num_workers=2)


imgs = loader_train.__iter__().next()[0].view(batch_size, 784)
show_images(imgs)

## Generating Random Noise

The first step is to generate uniform noise from -1 to 1 with shape `[batch_size, noise_dim]`

Hint: use `torch.rand`.

Implement `sample_noise` below.

In [ ]:
def sample_noise(batch_size, noise_dim, dtype=torch.float, device='cpu'):
  """
  Generate a PyTorch Tensor of uniform random noise.

  Input:
  - batch_size: Integer giving the batch size of noise to generate.
  - noise_dim: Integer giving the dimension of noise to generate.
  
  Output:
  - A PyTorch Tensor of shape (batch_size, noise_dim) containing uniform
    random noise in the range (-1, 1).
  """
  noise = None
  ##############################################################################
  #                           BEGINNING OF YOUR CODE                           #
  ##############################################################################
  ##############################################################################
  # TODO: Implement sample_noise.  
  # The generated noise values (from uniform distribution) must be in
  # the interval [-1,1].
  # However, "torch.rand" generates random values in [0,1].
  # For that, we must use a `transformation`, from [0,1] to [-1,1].
  # -----
  # The idea is given below (from: https://stackoverflow.com/a/44375813)
  # If U is a random variable uniformly distributed on [0,1],
  # then `(r1 - r2) * U + r2` is uniformly distributed on [r1,r2].
  # -----
  # In our case, `r1`=-1 and `r2`=1                                            
  ##############################################################################
  # Replace "pass" statement with your code
  pass
  ##############################################################################
  #                              END OF YOUR CODE                              #
  ##############################################################################

  return noise

## Generator and Discriminator

**The definitions of these functions are provided to you.**

They are explained below:

### Discriminator

The architecture is:
 * Fully connected layer with input size 784 and output size 256
 * LeakyReLU with alpha 0.01
 * Fully connected layer with input_size 256 and output size 256
 * LeakyReLU with alpha 0.01
 * Fully connected layer with input size 256 and output size 1
  
The output of the discriminator is of shape `[batch_size, 1]`, and contaisn real numbers corresponding to the scores that each of the `batch_size` inputs is a real image.

### Generator

The architecture is:
 * Fully connected layer from noise_dim to 1024
 * `ReLU`
 * Fully connected layer with size 1024 
 * `ReLU`
 * Fully connected layer with size 784
 * `TanH` (to clip the image to be in the range of [-1,1])

 This outputs a tensor of shape of `[batch_size, 784]` that is the image generated with the given sample noise to the generator.

In [ ]:
NOISE_DIM = 96 # default noise dimension

def discriminator():
  """
  Build and return a PyTorch nn.Sequential model implementing the architecture in the notebook.
  """

  model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 256),  # 1st Fully-Connected layer.
    nn.LeakyReLU(0.01),
    nn.Linear(256, 256),  # 2nd Fully-Connected layer.
    nn.LeakyReLU(0.01),
    nn.Linear(256, 1)     # 3rd Fully-Connected layer.
  )

  return model


def generator(noise_dim=NOISE_DIM):
  """
  Build and return a PyTorch nn.Sequential model implementing the architecture in the notebook.
  """
  model = nn.Sequential(
    nn.Linear(noise_dim, 1024),  # 1st Fully-Connected layer.
    nn.ReLU(),
    nn.Linear(1024, 1024),       # 2nd Fully-Connected layer.
    nn.ReLU(),
    nn.Linear(1024, 784),        # 3rd Fully-Connected layer.
    nn.Tanh()
  )

  return model  

## Loss functions

This is the most important part. Please read the instructions below carefully.

Compute the generator and discriminator loss. The generator loss is:
$$\ell_G  =  -\mathbb{E}_{z \sim p(z)}\left[\log D(G(z))\right]$$
and the discriminator loss is:
$$ \ell_D = -\mathbb{E}_{x \sim p_\text{data}}\left[\log D(x)\right] - \mathbb{E}_{z \sim p(z)}\left[\log \left(1-D(G(z))\right)\right]$$
We will be *minimizing* these losses.

For the purpose of these equations, we assume that the output from the discriminator is a real number in the range $0 < D(x) < 1$ which results from squashing the raw score from the discriminator through a sigmoid function. However for a cleaner and more numerically stable implementation, we have not included the sigmoid in the discriminator architecture above -- instead we will implement the sigmoid as part of the loss function.

**HINTS**: You can use the function [`torch.nn.functional.binary_cross_entropy_with_logits`](https://pytorch.org/docs/stable/nn.functional.html#binary-cross-entropy-with-logits) to compute these losses in a numerically stable manner.

Given a score $s\in\mathbb{R}$ and a label $y\in\{0, 1\}$, the binary cross entropy loss (with logits) is defined as:

$$ bce(s, y) = -y * \log(\sigma(s)) - (1 - y) * \log(1 - \sigma(s)) $$

where $\sigma(s)=1/(1+\exp(-s))$ is the sigmoid function.

A naive implementation of this formula can be numerically unstable, so you should prefer to use the built-in PyTorch implementation.

You will also need to compute labels corresponding to real or fake and use the logit arguments to determine their size. Make sure you cast these labels to the correct data type using the global `dtype` variable, for example:

`true_labels = torch.ones(size, device=device)`

Instead of computing the expectation of $\log D(G(z))$, $\log D(x)$ and $\log \left(1-D(G(z))\right)$, we will be averaging over elements of the minibatch, so make sure to combine the loss by averaging instead of summing.

In [ ]:
def discriminator_loss(logits_real, logits_fake):
  """
  Computes the discriminator loss described above.
  
  Inputs:
  - logits_real: PyTorch Tensor of shape (N,) giving scores for the real data.
  - logits_fake: PyTorch Tensor of shape (N,) giving scores for the fake data.
  
  Returns:
  - loss: PyTorch Tensor containing (scalar) the loss for the discriminator.
  """
  loss = None
  ##############################################################################
  #                           BEGINNING OF YOUR CODE                           #
  ##############################################################################
  ##############################################################################
  # TODO: Implement discriminator_loss.                                        #
  ##############################################################################

  # For the discriminator (D), the true target (y = 1) corresponds to "real" images.
  # Thus, for the scores of real images, the target is always 1 (a vector).
  real_labels = 
  # Compute the BCE for the scores of the real images.
  # Note that the BCE itself uses the Expectation formula (in addition, an average is
  # taken throughout the losses, not a sum [as requested in this assignment]).
  real_loss = 

  # For D, the false target (y = 0) corresponds to "fake" images.
  # Thus, for the scores of fake images, the target is always 0 (a vector).
  fake_labels = 
  # As for the real scores, compute the BCE loss for the fake images.
  fake_loss = 

  # Sum "real" and "fake" losses.
  # That is, BCE has already taken into account the "negated equation" form,
  # the "log" (in the Expectation) and the "mean" (instead on the "sum").
  loss = 

  ##############################################################################
  #                              END OF YOUR CODE                              #
  ##############################################################################
  return loss

def generator_loss(logits_fake):
  """
  Computes the generator loss described above.

  Inputs:
  - logits_fake: PyTorch Tensor of shape (N,) giving scores for the fake data.
  
  Returns:
  - loss: PyTorch Tensor containing the (scalar) loss for the generator.
  """
  loss = None
  ##############################################################################
  #                           BEGINNING OF YOUR CODE                           #
  ##############################################################################
  ##############################################################################
  # TODO: Implement generator_loss.                                            #
  ##############################################################################

  # For the generator (G), the true target (y = 1) corresponds to "fake" images.
  # Thus, for the scores of fake images, the target is always 1 (a vector).
  fake_labels =
  # Compute the BCE for the scores of the fake images.
  fake_loss =

  # The generator loss is "fake_loss".
  # That is, BCE has already taken into account the "negated equation" form,
  # the "log" (in the Expectation) and the "mean" (insetead on the "sum").
  loss =

  ##############################################################################
  #                              END OF YOUR CODE                              #
  ##############################################################################
  return loss

## Training

We provide you the main training loop... you won't need to change this function, but we encourage you to read through and understand it. 

In [ ]:
def run_a_gan(D, G, D_solver, G_solver, discriminator_loss, generator_loss, save_filename, show_every=250, 
              batch_size=128, noise_size=96, num_epochs=10):
  """
  Train a GAN!
  
  Inputs:
  - D, G: PyTorch models for the discriminator and generator
  - D_solver, G_solver: torch.optim Optimizers to use for training the
    discriminator and generator.
  - discriminator_loss, generator_loss: Functions to use for computing the generator and
    discriminator loss, respectively.
  - show_every: Show samples after every show_every iterations.
  - batch_size: Batch size to use for training.
  - noise_size: Dimension of the noise to use as input to the generator.
  - num_epochs: Number of epochs over the training dataset to use for training.
  """
  iter_count = 0
  for epoch in range(num_epochs):
    for x, _ in loader_train:
      if len(x) != batch_size:
        continue
      D_solver.zero_grad()
      real_data = x.view(-1, 784).to(device)
      logits_real = D(2* (real_data - 0.5))

      g_fake_seed = sample_noise(batch_size, noise_size, dtype=real_data.dtype, device=real_data.device)
      fake_images = G(g_fake_seed).detach()
      logits_fake = D(fake_images)

      d_total_error = discriminator_loss(logits_real, logits_fake)
      d_total_error.backward()        
      D_solver.step()

      G_solver.zero_grad()
      g_fake_seed = sample_noise(batch_size, noise_size, dtype=real_data.dtype, device=real_data.device)
      fake_images = G(g_fake_seed)

      gen_logits_fake = D(fake_images)
      g_error = generator_loss(gen_logits_fake)
      g_error.backward()
      G_solver.step()

      if (iter_count % show_every == 0):
        print('Iter: {}, D: {:.4}, G:{:.4}'.format(iter_count,d_total_error.item(),g_error.item()))
        imgs_numpy = fake_images.data.cpu()#.numpy()
        show_images(imgs_numpy[0:16])
        plt.show()
        print()
      iter_count += 1
    if epoch == num_epochs - 1:
      show_images(imgs_numpy[0:16])

Now run the cell below to train your first GAN! Your last epoch results will be stored in `fc_gan_results.jpg` for visualization.

In the iterations in the low 100s you should see black backgrounds, fuzzy shapes as you approach iteration 1000, and decent shapes, about half of which will be sharp and clearly recognizable as we pass 3000.

In [ ]:
set_seed(0)

# Make the discriminator
D = discriminator().to(device)

# Make the generator
G = generator().to(device)

# Use the function you wrote earlier to get optimizers for the Discriminator and the Generator
D_solver = optim.Adam(D.parameters(), lr=1e-3, betas=(0.5, 0.999))
G_solver = optim.Adam(G.parameters(), lr=1e-3, betas=(0.5, 0.999))
# Run it!
run_a_gan(D, G, D_solver, G_solver, discriminator_loss, generator_loss, 'fc_gan_results.jpg')

## Question 5

What is the discriminator error after 4500 iterations displayed in the cell? (choose the closest answer)

1. 1.303
2. 1.450
3. 0.352
4. 0.298

In [ ]:
set_seed(4)

g_fake_seed = sample_noise(64, 96, dtype=dtype, device=device)
fake_images = G(g_fake_seed)

gen_logits_fake = D(fake_images)
g_error = generator_loss(gen_logits_fake)

print(g_error.item())

## Question 6

What is the generator error displayed in the cell? (choose the closest answer)

1. 0.7431
2. 0.8152
3. 0.7815
4. 0.7611


# Normalizing Flows [Optional]


<script type="text/javascript" async
src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/MathJax.js?config=TeX-MML-AM_CHTML">
</script>

<script type="text/x-mathjax-config">
    MathJax.Hub.Config({
    tex2jax: {
    inlineMath: [ ['$','$'], ["\\(","\\)"] ],
    processEscapes: true
    }
    });
</script>

<p><em>Normalizing flows</em> provide a mechanism to transform simple distributions into more complex ones without sacrificing the computational conveniences that make the former appealing and practical.</p>
<p>The idea was introduced in its popular form to the machine learning community in <a href="https://arxiv.org/abs/1505.05770">Rezende et al.</a></p>



<h2 id="normalizing-flows">Overview</h2>
<p>The basic idea is to pass a sample from a simple base distribution through a series of transformations or <em>flows</em>. The transformations $f$ must be smooth, differentiable ($f$ and $f^-1$) and invertible. Functions of this type are known as diffeomorphisms. The transformed sample can now be thought of as a sample from a more complex distribution. The invertibility condition gives a closed form description of the relationship between the density of the initial distribution and the final.</p>
<p>Consider an invertible smooth mapping $$f: \mathcal{R}^{d} \to \mathcal{R}^d$$ with an inverse $f^{-1} = g$.</p>
<p>Given a random variable $\mathbb{z}$ with distribution $q(\mathbb{z})$, $\mathbb{z}^{'} = f(\mathbb{z})$ has the distribution:</p>
<p>$$q(\mathbb{z}') = q(\mathbb{z}) |det \frac{\partial f^{-1}}{\partial \mathbb{z}^{'}} | = q(\mathbb{z})| det \frac{\partial f}{\partial \mathbb{z}} |^{-1}$$</p>
<p>Additionally if we use $k$ flows and transform our initial sample $\mathbb{z}_0$: $\mathbb{z}_k= f_K \circ … \circ f_2 \circ f_1(\mathbb{z}_0)$</p>
<p>We can get the final log-density $\ln(q_k(\mathbb{z}))$ as follows:</p>
<p>$\ln q_K(\mathbb{z}_K)= \ln_{q_0}(\mathbb{z}_0)-\sum_{k=1}^K \ln \lvert det \frac{\partial f_k}{\partial \mathbb{z}_{{k-1}}}\rvert$</p>
<p>Nice!</p>
<p>Now all we need is an invertible transformation to use. The authors propose a family of transformations called <em>planar flows</em> which take the form: $$f(z)=z+uh(w^T z + b)$$ Where:</p>
<ul>
<li>$\mathbb{w} \in \mathcal{R}^D$</li>
<li>$\mathbb{u} \in \mathcal{R}^D$</li>
<li>$b \in \mathcal{R}$</li>
<li>$h$ is an elementwise nonlinearity (like <em>tanh</em>).</li>
</ul>
<p>The Determinant-Jacobian can be computed efficiently:</p>
<p>$$\psi(\mathbb{z}) = h’(\mathbb{w}^Tz + b)\mathbb{w}$$ $$| det \frac{\partial f}{\partial \mathbb{z}}| = \lvert 1+ \mathbb{u}^T \psi(\mathbb{z}) \rvert $$</p>
<p>Our formula for the final log density is:</p>
<p>$$\ln q_K(\mathbb{z}_K) = \ln_{q_0}(\mathbb{z}) - \sum_{k=1}^K \ln \lvert 1 + \mathbb{u}_k^T \psi_k(\mathbb{z}_{k-1})\rvert$$</p>

In [ ]:
# Do not modify this code

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)  

set_seed(0)

In [ ]:
class PlanarFlow(nn.Module):
    """
    A single planar flow, computes T(x) and log(det(jac_T)))
    """
    def __init__(self, D):
        super(PlanarFlow, self).__init__()
        self.u = nn.Parameter(torch.Tensor(1, D), requires_grad=True)
        self.w = nn.Parameter(torch.Tensor(1, D), requires_grad=True)
        self.b = nn.Parameter(torch.Tensor(1), requires_grad=True)
        self.h = torch.tanh
        self.init_params()

    def init_params(self):
        self.w.data.uniform_(-0.01, 0.01)
        self.b.data.uniform_(-0.01, 0.01)
        self.u.data.uniform_(-0.01, 0.01)

    def forward(self, z):

        f_output = None

        ################################################################################################
        #                                  BEGINNING OF YOUR CODE                                      #  
        ################################################################################################
        ################################################################################################
        # TODO: Implement the forward pass denoted by f(z). Store the result in f_output.              #
        ################################################################################################
        # Replace "pass" statement with your code
        pass

        # matrix-multiply z and w, and add b
        linear_term = 
        # add z to u * h(linear_term)
        f_output = 

        ################################################################################################
        #                                     END OF YOUR CODE                                         #
        ################################################################################################
        return f_output

    def h_prime(self, x):
        """
        Derivative of tanh
        """
        return (1 - self.h(x) ** 2)

    def psi(self, z):

        psi_output = None

        ################################################################################################
        #                                  BEGINNING OF YOUR CODE                                      #  
        ################################################################################################
        ################################################################################################
        # TODO: Implement the function psi(z). Store the result in psi_output.                         #
        ################################################################################################
        # Replace "pass" statement with your code
        pass

        # matrix-multiply z and w, add b
        inner = 

        # multiply w to h_prime(inner)
        psi_output = 

        ################################################################################################
        #                                     END OF YOUR CODE                                         #
        ################################################################################################
        return psi_output
        

    def log_det(self, z):

        log_det_output = None

        ################################################################################################
        #                                  BEGINNING OF YOUR CODE                                      #  
        ################################################################################################
        ################################################################################################
        # TODO: Implement the logarithm of the determinant Jacobian.                                   #
        # Store the result in log_det_output.                                                          #
        # Note: Take the absolute value of the determinant, before applying logarithm                  #
        ################################################################################################
        # Replace "pass" statement with your code
        pass

        # calculate inner as 1 + matrix-multiply(psi(z), u)
        inner = 

        # calculate log_det_output as log(|inner|)
        log_det_output = 
        ################################################################################################
        #                                     END OF YOUR CODE                                         #
        ################################################################################################

        return log_det_output

In [ ]:
class NormalizingFlow(nn.Module):
    """
    A normalizing flow composed of a sequence of planar flows.
    """
    def __init__(self, D, n_flows=2):
        super(NormalizingFlow, self).__init__()
        self.flows = nn.ModuleList(
            [PlanarFlow(D) for _ in range(n_flows)])

    def sample(self, base_samples):
        """
        Transform samples from a simple base distribution
        by passing them through a sequence of Planar flows.
        """
        samples = base_samples
        for flow in self.flows:
            samples = flow(samples)
        return samples

    def forward(self, x):
        """
        Computes and returns the sum of log_det_jacobians
        and the transformed samples T(x).
        """
        sum_log_det = 0
        transformed_sample = x

        for i in range(len(self.flows)):
            log_det_i = (self.flows[i].log_det(transformed_sample))
            sum_log_det += log_det_i
            transformed_sample = self.flows[i](transformed_sample)

        return transformed_sample, sum_log_det


In [ ]:
"""
Potential functions U(x) from Rezende et al. 2015
p(z) is then proportional to exp(-U(x)).
Since we log this value later in the optimized bound,
no need to actually exp().
"""


def w_1(z):
    return torch.sin((2 * math.pi * z[:, 0]) / 4)


def w_2(z):
    return 3 * torch.exp(-.5 * ((z[:, 0] - 1) / .6) ** 2)


def sigma(x):
    return 1 / (1 + torch.exp(- x))


def w_3(z):
    return 3 * sigma((z[:, 0] - 1) / .3)


def pot_1(z):
    z_1, z_2 = z[:, 0], z[:, 1]
    norm = torch.sqrt(z_1 ** 2 + z_2 ** 2)
    outer_term_1 = .5 * ((norm - 2) / .4) ** 2
    inner_term_1 = torch.exp((-.5 * ((z_1 - 2) / .6) ** 2))
    inner_term_2 = torch.exp((-.5 * ((z_1 + 2) / .6) ** 2))
    outer_term_2 = torch.log(inner_term_1 + inner_term_2 + 1e-7)
    u = outer_term_1 - outer_term_2
    return - u


def pot_2(z):
    u = .5 * ((z[:, 1] - w_1(z)) / .4) ** 2
    return - u


def pot_3(z):
    term_1 = torch.exp(-.5 * (
        (z[:, 1] - w_1(z)) / .35) ** 2)
    term_2 = torch.exp(-.5 * (
        (z[:, 1] - w_1(z) + w_2(z)) / .35) ** 2)
    u = - torch.log(term_1 + term_2 + 1e-7)
    return - u


def pot_4(z):
    term_1 = torch.exp(-.5 * ((z[:, 1] - w_1(z)) / .4) ** 2)
    term_2 = torch.exp(-.5 * ((z[:, 1] - w_1(z) + w_3(z)) / .35) ** 2)
    u = - torch.log(term_1 + term_2)
    return - u

In [ ]:
set_seed(0)

def random_normal_samples(n, dim=2):
    return torch.zeros(n, dim).normal_(mean=0, std=1)

# we are using 16 planar flows
model = NormalizingFlow(2, 16)

# target density as pot1
target_density = pot_1

# RMSprop is what they used in renzende et al
opt = torch.optim.RMSprop(
    params=model.parameters(),
    lr=1e-4,
    momentum=0.9
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', patience=1000)
losses = []

for iter_ in range(10000):
    if iter_ % 100 == 0:
        print("Iteration {}".format(iter_), end=": ")

    samples = Variable(random_normal_samples(100))

    z_k, sum_log_det = model(samples)
    log_p_x = target_density(z_k)

    # Reverse KL since we can evaluate target density but can't sample
    loss = (- sum_log_det - (log_p_x)).mean()

    opt.zero_grad()
    loss.backward()
    opt.step()
    scheduler.step(loss)

    losses.append(loss.item())

    if iter_ % 100 == 0:
        print("Loss {}".format(loss.item())) 

In [ ]:
set_seed(0)

samples = Variable(random_normal_samples(100))

z_k, sum_log_det = model(samples)
log_p_x = target_density(z_k)

print(sum_log_det.mean().item())